![QF-logo](https://brainhelper.wordpress.com/wp-content/uploads/2025/03/qf-up.png)

# Quantum Gate Decomposition via Lie Theory
### Khaneja-Glaser Algorithm and Quantum Machine Learning (QML) with PennyLane
Brian Hepler | Quantum Formalism

### Objectives
In this notebook, we explore two key methods for decomposing arbitrary two-qubit gates in $SU(4)$:

- **Khaneja-Glaser Decomposition**:  
  An explicit, exact decomposition method based on the Cartan (KAK) decomposition from Lie Theory [1].

- **Quantum Machine Learning (QML)**:  
  A gradient-based variational approach utilizing PennyLane's automatic differentiation and variational quantum circuits to approximate the desired unitary gates.

**Learning Goals:**
- Gain intuition and computational experience with gate decomposition algorithms used extensively in quantum computing.
- Develop familiarity with practical numerical implementations of these algorithms.
- Understand connections between abstract mathematical concepts (Lie groups/algebras) and modern quantum computing applications.

## Part 1: Khaneja-Glaser Algorithm Implementation
---

As mentioned above, the **Khaneja-Glaser algorithm** is a specific implementation of the **Cartan (KAK) decomposition**, allowing us to factor an arbitrary two-qubit gate $U \in SU(4)$ as:

$$
U = k_1 A k_2
$$

Here,

- $k_1, k_2 \in SU(2)\otimes SU(2)$ represent **local** gates acting independently on each qubit.
- $A$ is an entangling, **nonlocal** two-qubit gate characterized explicitly by parameters associated with a Cartan subalgebra of the Lie algebra $\mathfrak{su}(4)$.


Let's briefly recall how the KAK decomposition works. Let $G$ be a compact, connected Lie group, and let $\mathfrak{g}$ be its associated Lie algebra. 

The KAK decomposition on $G$ begins a Cartan decomposition of $\mathfrak{g}$, which is an orthogonal splitting 
$$
\mathfrak{g} = \mathfrak{l} \oplus \mathfrak{p}
$$
into a *vertical* subspace $\mathfrak{l}$ and a *horizontal* subspace $\mathfrak{p}$. Here, the splitting is orthogonal with respect to the Killing form on $\mathfrak{g}$, but in the case of $\mathfrak{g} = \mathfrak{su}(4)$, we can simply take the **Hilbert-Schmidt inner product**, defined by $\langle x,y \rangle = \mathrm{Tr}(x^* y)$, where $x^* = \overline{x}^T$ is conjugate transpose of $x$. The subspaces $\mathfrak{l}$ and $\mathfrak{p}$ satisfy the following important commutation relations:
$$
[\mathfrak{l},\mathfrak{l}] \subset \mathfrak{l}, \quad [\mathfrak{l},\mathfrak{p}] \subset \mathfrak{p}, \quad [\mathfrak{p},\mathfrak{p}] \subset \mathfrak{l}.
$$
Together, these imply that 
1. The vertical subspace $\mathfrak{l}$ is a Lie subalgebra of $\mathfrak{g}$,
2. The horizontal subspace $\mathfrak{p}$ is invariant under the adjoint action of $\mathfrak{l}$, and
3. Any Lie subalgebra $\mathfrak{a} \subset \mathfrak{p}$ must be Abelian, i.e., $[\mathfrak{a},\mathfrak{a}] = 0$.

For $\mathfrak{su}(4)$, we take $\mathfrak{l} = \mathfrak{su}(2) \oplus \mathfrak{su}(2)$ as infinitesimal generators of our local gates, and the complement 
$$
\mathfrak{p} = \mathrm{Span} \{iX \otimes X, iY \otimes Y, iZ \otimes Z, iX \otimes Y, iY \otimes Z, \cdots \}
$$
as infinitesimal generators for our entangling gates. 

Let $\mathfrak{a} \subset \mathfrak{p}$ be a Lie subalgebra of $\mathfrak{g}$ which is maximal with respect to inclusion. This maximal Abelian subalgebra is not unique -- however, if $\mathfrak{a}$ is any other such maximal Abelian subalgebra of $\mathfrak{p}$, there exists an element $k \in K = \exp(\mathfrak{l})$ such that 
$$
\mathfrak{a}' = k \mathfrak{a} k^{-1} = \mathrm{Ad}_k (\mathfrak{a}).
$$
That is, $\mathfrak{a}$ and $\mathfrak{a}'$ are *conjugate* under the Adjoint action of the Lie subgroup $K \subset G$. These conjugacy classes (or orbits of the Adjoint action of $K$) comprise all of $\mathfrak{p}$, i.e.,
$$
\mathfrak{p} = \bigcup_{k \in K} \mathrm{Ad}_k(\mathfrak{a})
$$
for any choice of maximal Abelian subalgebra. For simplicity of notation, we often just refer to such a subalgebra as a **Cartan subalgebra**. A standard choice of Cartan subalgebra for $\mathfrak{su}(4)$ is generated by the Pauli matrices:
$$
\mathfrak{a} = \mathrm{Span} \{i X \otimes X, i Y \otimes Y, i Z \otimes Z \}.
$$
A quick check shows that these elements all commute with one another!

In [1]:
# Import libraries for Khaneja-Glaser implementation
import numpy as np 
from scipy.linalg import expm, logm, svd, norm
from scipy.optimize import root

# Define Pauli matrices
X = np.array([[0, 1], [1, 0]]) # Pauli-X
Y = np.array([[0, -1j], [1j, 0]]) # Pauli-Y
Z = np.array([[1, 0], [0, -1]]) # Pauli-Z
I2 = np.eye(2) # 2x2 identity

In [2]:
# Define basis for the Cartan subalgebra for su(4): span{XX, YY, ZZ}.
XX = np.kron(X, X) 
YY = np.kron(Y, Y)
ZZ = np.kron(Z, Z)
cartan_basis = [XX, YY, ZZ]

^By linearity, we can omit the factor of $i$ in front of each tensor product when checking commutativity.

How does this help us decompose elements in our original Lie group $SU(4)$? Since $SU(4)$ is compact and connected, the exponential map is surjective, so any Lie group element $U \in SU(4)$ can be expressed as $U = e^g$ for some $g \in \mathfrak{su}(4)$. 

To achieve this decomposition, we implement the following steps, based on the approach in Sá Earp–Pachos [2] and the Cartan decomposition method developed by Khaneja–Glaser [1]:

### 1. **Compute the logarithm** $g = \log(U)$ to map $U$ into the Lie algebra.

We decompose $\mathfrak{su}(4) = \mathfrak{l} \oplus \mathfrak{p}$, where $\mathfrak{l}$ is the Lie algebra of local (single-qubit) operations and $\mathfrak{p}$ contains non-local directions. We aim to write $g = l + p$, with $l \in \mathfrak{l}$, $p \in \mathfrak{p}$.


### 2. **Project $g$ orthogonally onto a fixed Cartan subalgebra** $\mathfrak{a} \subset \mathfrak{p}$.

This gives an initial approximation for the non-local entangling part of $g$, denoted $p_{\text{approx}}$. We use a fixed basis of $\mathfrak{a} = \mathrm{Span}\{i\,XX, i\,YY, i\,ZZ\}$ for this projection.


### 3. **Numerical refinement** of $p_{\text{approx}}$ using a root-finding method.

We refine $p_{\text{approx}}$ to a more accurate solution $p \in \mathfrak{a}$ using the **Baker–Campbell–Hausdorff (BCH) expansion** of the group product $e^l e^p$. The BCH series gives:

$$
g = l + p + \frac{1}{2}[l,p] + \frac{1}{12}[l,[l,p]] + \frac{1}{12}[p,[p,l]] + \cdots
$$

By truncating after the leading terms, we derive an approximation for $l$ as:

$$
l_{\text{approx}} = g - p_{\text{approx}} - \tfrac{1}{2}[g - p_{\text{approx}},\, p_{\text{approx}}].
$$

We then define a **residual function** that measures the deviation of $p$ from satisfying this truncated relation, and use a numerical root solver to drive this residual to zero.

> **Why this works:** This step numerically solves a fixed-point condition derived in Khaneja–Glaser’s decomposition framework, approximating the condition that $g = l + p$ with higher-order corrections. The residual we minimize corresponds to the leading terms in the BCH expansion and enforces the consistency of $p \in \mathfrak{a}$ and $l \in \mathfrak{l}$ in reconstructing $g$. (see [1])


### 4. **Extract local factors** from the decomposition.

Having found the refined value of $p$ (denoted `p_sol` in the code), we recover:

$$
e^l = U e^{-p} = K_0.
$$

Next, we diagonalize $p$ to obtain a canonical form. Since $p \in \mathfrak{a}$ is Hermitian, there exists a unitary matrix $K_1 \in SU(4)$ such that

$$
a = K_1^{-1} p K_1
$$

is diagonal. Then:

$$
e^p = K_1 \exp(a) K_1^{-1}, \quad \text{with } A = \exp(a).
$$

> **Clarification:** The diagonal matrix $a$ lies in a Cartan subalgebra conjugate to the original $\mathfrak{a} = \mathrm{Span}\{iXX, iYY, iZZ\}$. While we project into a fixed Cartan basis, the actual decomposition ends up expressed in the eigenbasis of $p$, which is equally valid. All Cartan subalgebras in $\mathfrak{su}(4)$ are conjugate, and so this does not affect the correctness of the decomposition.


### 5. **Final approximation**

We now obtain the decomposition:

$$
U_{\text{approx}} = K_0 K_1 A K_1^{-1}
$$

and quantify its accuracy using the Hilbert–Schmidt norm:

$$
\text{Error} = \| U - U_{\text{approx}} \|_{\text{HS}}.
$$

> **Note on canonical form:** One can further simplify the result by sorting the eigenvalues of $a$ (placing it into the Weyl chamber) and fixing the global phases of the eigenvectors in $K_1$, but this is not necessary for correctness. The decomposition as given is valid and reconstructs $U$ to machine precision (observed error $\sim 3.3 \times 10^{-16}$).

---

Here’s the complete numerical implementation:

In [3]:
def project_onto_cartan(A, basis):
    """
    Projects matrix A onto the subspace spanned by the given basis elements.
    Assumes the basis is orthogonal (or normalized) with respect to the Frobenius inner product.
    """
    proj = np.zeros_like(A, dtype=complex)
    for B in basis:
        coef = np.trace(A.conj().T @ B) / np.trace(B.conj().T @ B)
        proj += coef * B
    return proj

In [4]:
def cartan_decomposition(U, tol=1e-6):
    # Step 1: Compute the matrix logarithm of U.
    g = logm(U)
    
    # Step 2: Project g onto the Cartan subalgebra to obtain an initial candidate for p.
    p_approx = project_onto_cartan(g, cartan_basis)
    
    # Step 3a: Define a residual function that operates on real-valued vectors.
    def residual_real(vec):
        # Reconstruct the complex matrix P from its real and imaginary parts.
        P = vec[:16].reshape((4, 4)) + 1j * vec[16:].reshape((4, 4))
        commutator = g @ P - P @ g
        l_approx = g - P - 0.5 * commutator
        proj = project_onto_cartan(g - l_approx, cartan_basis)
        res = P - proj
        # Separate into real and imaginary parts and flatten.
        return np.concatenate((np.real(res).flatten(), np.imag(res).flatten()))
    
    # Step 3b: Create an initial guess by splitting p_approx into real and imaginary parts.
    vec0 = np.concatenate((np.real(p_approx.flatten()), np.imag(p_approx.flatten())))
    sol = root(residual_real, vec0, tol=tol)
    if not sol.success:
        raise ValueError("Root-finding for p did not converge.")
    # Step 3c: Reconstruct p from the solution vector.
    p_sol = sol.x[:16].reshape((4, 4)) + 1j * sol.x[16:].reshape((4, 4))
    
    # Step 4a: Compute K0 = U * exp(-m_sol)
    K0 = U @ expm(-p_sol)
    
    # Step 4b: For simplicity, we use the eigen-decomposition of p_sol to define K1.
    eigvals, eigvecs = np.linalg.eig(p_sol)
    K1 = eigvecs  # In a full algorithm, ordering and phase corrections would be applied.

    # Step 4c: Compute a = K1^(-1) * p_sol * K1 and then A = exp(a)
    a = np.linalg.inv(K1) @ p_sol @ K1
    A = expm(a)
    
    # Step 5: Reconstruct U as a check: U_reconstructed = K0 * K1 * A * K1^{-1}
    U_reconstructed = K0 @ K1 @ A @ np.linalg.inv(K1)

    
    error = norm(U - U_reconstructed)
    print("Reconstruction error:", error)
    
    return U_reconstructed, K0, K1, A, error

### Numerical Verification

To demonstrate the algorithm, we generate a random $SU(4)$ matrix using scipy.stats, which samples a unitary matrix uniformly at random from $U(4)$ according to the **Haar measure**. After normalizing our random unitary, we verify our decomposition by checking how accurately we can reconstruct the original matrix:

In [7]:
from scipy.stats import unitary_group

U = unitary_group.rvs(4) # generates a random element U of U(4)
det_U = np.linalg.det(U) # divide U by its determinant to normalize
U_SU = U / det_U**(1/4) # U_SU is now a random element of SU(4)

# Test the decomposition.
U_reconstructed ,K0, K1, A, err = cartan_decomposition(U_SU)

Reconstruction error: 3.3972246307450016e-16


### Interpretation and Applications

The Cartan decomposition explicitly separates the “local” structure (represented by $k_0$ and $k_1$, which can be implemented via single-qubit gates) from the “nonlocal” or entangling operations (represented by $A$). Such decompositions are crucial in quantum computing for efficient gate synthesis, quantum circuit optimization, and characterization of quantum entanglement.

--- 

## Part 2: Quantum Machine Learning Approach

We now demonstrate an alternative approach to performing the KAK decomposition for a two‑qubit unitary in $SU(4)$. Rather than computing a closed‑form solution, we build a **variational quantum circuit** whose structure reflects the canonical form

$$
U = (U_A \otimes U_B)\; \exp\Bigl[-\frac{i}{2}(c_x\,X\otimes X + c_y\,Y\otimes Y + c_z\,Z\otimes Z)\Bigr]\; (V_A \otimes V_B)
$$

and use **gradient descent** (with Pennylane’s automatic differentiation) to optimize the parameters so that the circuit unitary approximates a given target $U_\text{target}$. 

### Why QML? 
While exact algebraic methods like Khaneja-Glaser are rigorous and precise, they're not always easy to implement efficiently on quantum hardware. Quantum machine learning methods, especially variational algorithms, are practical alternatives for synthesizing gates approximately.

### Variational Quantum Circuit Ansatz

We implement a variational quantum circuit designed to approximate arbitrary two-qubit gates $U \in SU(4)$. The circuit structure (called its *ansatz*) consists of three distinct layers:

1. **Initial local rotations:**
Two single-qubit gates (parameterized by **Euler angles**) acting independently on each qubit. Each gate is implemented using the universal single-qubit rotation gate $U(\theta, \phi, \lambda)$:
$$
U(\theta, \phi, \lambda) = R_Z(\phi) R_Y(\theta) R_Z(\lambda)
$$

where $R_Z(\phi) = e^{-\frac{i}{2} \phi Z}$, etc. are rotations about the $Z$ or $Y$ axes. This gives 6 parameters, 
$$
U_{local}^{(pre)}(\theta_0,\theta_1,\theta_2,\theta_3,\theta_4,\theta_5) = U(\theta_0,\theta_1,\theta_2) \otimes U(\theta_3,\theta_4,\theta_5).
$$

2. **Nonlocal (Entangling) layer:**
Parameterized two-qubit gates that generate entanglement between qubits. We use three canonical two-qubit rotations:
$$
R_{XX}(c_x) = e^{-\frac{i}{2} c_x X\otimes X}, \quad R_{YY}(c_y) = e^{-\frac{i}{2} c_y Y \otimes Y}, \quad R_{ZZ}(c_z) = e^{-\frac{i}{2} c_z Z \otimes Z}
$$

These correspond exactly to the KAK decomposition’s entangling component. This contributes 3 parameters, 
$$
U_{nonlocal}(\theta_6,\theta_7,\theta_8) = R_{XX}(\theta_6) R_{YY}(\theta_7) R_{ZZ}(\theta_8).
$$
Since these elements arise from the Cartan subalgebra of $\mathfrak{su}(4)$, they commute with each other, and thus their exponentials multiply in the "expected" way. I.e., we use the result that if $A,B$ are commuting Hermitian operators, then $e^A e^B = e^{A+B}$.

3. **Final Local Rotations:**
Another round of single-qubit rotations to further refine and approximate the desired target unitary. This last adjustment contributes another 6 parameters as in step 1, yielding the unitary $U_{local}^{(post)}$ with parameters $\theta_{10},\cdots,\theta_{14}$.

The final structure of circuit reflects the decomposition discussed in lecture:
$$
U_{\text{ansatz}}(\theta_0,\cdots,\theta_{14}) = (U_{\text{local}}^{(post)})\,  U_{\text{nonlocal}} \, (U_{\text{local}}^{(pre)})
$$

In [8]:
import pennylane as qml
import pennylane.numpy as np
import numpy as np_orig
from scipy.stats import unitary_group

# Generate a random 4x4 unitary from the Haar measure using SciPy
# and ensure it lies in SU(4) (i.e. determinant = 1 up to global phase).
U_target_np = unitary_group.rvs(4)
det = np_orig.linalg.det(U_target_np)
U_target_np = U_target_np / (det ** (1/4))
# Convert to Pennylane's numpy array
U_target = np.array(U_target_np)

print("Target unitary U_target:")
print(U_target)

Target unitary U_target:
[[ 0.17784254+0.53566725j -0.57679714-0.38914519j  0.32294883+0.14579992j
   0.09918299-0.24882299j]
 [ 0.34282511+0.08147512j  0.03338286-0.28824379j -0.26999841-0.62474674j
   0.50155264+0.2772565j ]
 [-0.3096699 +0.65474584j -0.12809739+0.45443503j -0.48434308-0.10585033j
  -0.05236295+0.06291185j]
 [ 0.15899801-0.08600837j -0.28295673-0.35873044j -0.38203116+0.13963697j
  -0.58842656+0.49686824j]]


In [92]:
# We use 15 parameters:
# - First 6 parameters: local rotations on qubits 0 and 1 (pre-nonlocal part)
# - Next 3 parameters: nonlocal interaction parameters (c_x, c_y, c_z)
# - Last 6 parameters: local rotations on qubits 0 and 1 (post-nonlocal part)

dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev, interface="autograd")
def variational_circuit(params):
    # Pre-nonlocal local rotations:
    qml.Rot(params[0], params[1], params[2], wires=0)
    qml.Rot(params[3], params[4], params[5], wires=1)
    
    # Nonlocal part implemented as a product of three Ising interactions.
    # Pennylane's IsingXX, IsingYY, IsingZZ gates implement:
    #   exp(-i * theta/2 * (X⊗X)), etc.
    # Setting theta = c_x (for example) gives the desired factor exp(-i/2 c_x X⊗X)
    qml.IsingXX(params[6], wires=[0, 1])
    qml.IsingYY(params[7], wires=[0, 1])
    qml.IsingZZ(params[8], wires=[0, 1])
    
    # Post-nonlocal local rotations:
    qml.Rot(params[9], params[10], params[11], wires=0)
    qml.Rot(params[12], params[13], params[14], wires=1)
    
    # We return the state for completeness.
    # (However, for our loss function we will extract the full unitary.)
    return qml.state()

In [94]:
def loss(params):
    # Get the unitary representation of the circuit.
    # qml.matrix returns the full unitary of the QNode (up to a global phase).
    U_est = qml.matrix(variational_circuit)(params)
    
    # Define a fidelity measure between unitaries (up to global phase):
    # Fidelity F = |Tr(U_target† U_est)| / 4.
    # Our loss is then 1 - F^2, which is 0 when U_est = U_target up to global phase.
    fidelity = np.abs(np.trace(np.conj(U_target).T @ U_est)) / 4
    return 1 - fidelity**2

### Optimization Procedure (Gradient Descent)

We'll now use PennyLane's **automatic differentiation** capabilities to optimize our variational ansatz, minimizing a suitable loss function to approximate our randomly generated $SU(4)$ gate.

**Our optimization process consists of these key steps:**

1. Define a parametrized quantum circuit (called an *ansatz*).
2. Choose a suitable cost function (the square of the Hilbert-Schmidt norm of $U_{target}-U_{est}$).
3. Perform gradient-descent steps iteratively updating the parameters to minimize our cost.
4. Monitor convergence and verify the accuracy of approximation numerically.

Let's implement this explicitly now.

In [110]:
# Initialize the 15 parameters uniformly between 0 and 2*pi.
params = np.random.uniform(0, 2 * np.pi, 15)

# Choose an optimizer (Pennylane's gradient descent).
opt = qml.GradientDescentOptimizer(stepsize=0.1)
steps = 200

for i in range(steps):
    params, current_loss = opt.step_and_cost(loss, params)
    if i % 10 == 0:
        print(f"Step {i:3d}: Loss = {current_loss:.6f}")

# After training, extract the learned unitary.
U_learned = qml.matrix(variational_circuit)(params)
final_fidelity = np.abs(np.trace(np.conj(U_target).T @ U_learned)) / 4
print("\nFinal fidelity (variational approach):", final_fidelity)

Step   0: Loss = 0.971404
Step  10: Loss = 0.958004
Step  20: Loss = 0.939135
Step  30: Loss = 0.914274
Step  40: Loss = 0.884007
Step  50: Loss = 0.849181
Step  60: Loss = 0.808309
Step  70: Loss = 0.754495
Step  80: Loss = 0.675598
Step  90: Loss = 0.564393
Step 100: Loss = 0.433813
Step 110: Loss = 0.308422
Step 120: Loss = 0.205466
Step 130: Loss = 0.134683
Step 140: Loss = 0.094060
Step 150: Loss = 0.072714
Step 160: Loss = 0.061215
Step 170: Loss = 0.054384
Step 180: Loss = 0.049762
Step 190: Loss = 0.046197

Final fidelity (variational approach): 0.9781936476814603


### Discussion

- **Variational Approach:**  
  By parameterizing a circuit that mimics the KAK form, we used gradient descent to optimize the circuit so that its unitary $U_{\text{learned}}$ approximates our target $U_{\text{target}}$. The loss function was defined via a fidelity measure that is insensitive to global phase.

- **Comparison:**  
  The closed-form KAK decomposition, when computed exactly, should reproduce the target unitary exactly (fidelity 1 up to numerical error). In our variational method, if the optimization converges well the final fidelity should be very close to 1.  

- **Advantages:**  
  This gradient-based (variational) method is fully differentiable and can be integrated into larger quantum machine learning pipelines. It also has the potential to “learn” decompositions when analytical methods are intractable or when one wants to optimize over additional cost functions.

- **Limitations:**  
  The optimization is iterative and may require careful tuning of hyperparameters (like the learning rate and the number of steps) and might get stuck in local minima. In contrast, a closed-form method computes the parameters directly.

Overall, this example demonstrates that with Pennylane’s QML libraries you can trade an analytical procedure for a differentiable, optimization‑based approach that fits naturally into variational quantum algorithms.

## Comparing Approaches
We've demonstrated two powerful yet distinct methodologies for decomposing two-qubit gates. The Khaneja-Glaser method explicitly utilizes Lie-theoretic decomposition, while the QML approach leverages modern automatic differentiation frameworks for variational optimization.

This comparison highlights the flexibility and power of integrating advanced mathematics with cutting-edge quantum computing techniques.

# Conclusion and Key Takeaways

In this notebook, you've gained hands-on experience with essential gate decomposition methods widely used in quantum computing:

- You've explicitly applied sophisticated Lie algebra theory via the Khaneja-Glaser algorithm.
- You've implemented modern quantum machine learning techniques with PennyLane to variationally approximate gates.

This demonstrates clearly how theoretical mathematics, specifically Lie theory, informs practical quantum computing tasks—skills highly valued in both research and industrial roles.

## References

1.  Khaneja, N., & Glaser, S. J. (2001). Cartan decomposition of $SU(2)$ and control of spin systems. Chemical Physics, 267(1), 11–23. DOI: [10.1016/S0301-0104(01)00318-4](https://doi.org/10.1016/S0301-0104(01)00318-4)


2. Henrique N. Sá Earp and Jiannis K. Pachos, A constructive algorithm for the Cartan decomposition of SU(2ⁿ), Journal of Mathematical Physics 46, 082108 (2005). DOI: [10.1063/1.2008210](https://doi.org/10.1063/1.2008210)

**Copyright © 2025 Quantum Formalism Academy. All rights reserved.**

This notebook is a product of **Quantum Formalism Academy** and is intended for educational purposes. Redistribution, modification, or commercial use of this material without prior written permission from Quantum Formalism is prohibited.

![QF-mission](https://brainhelper.wordpress.com/wp-content/uploads/2025/03/qf-down.png)